# G2Engine Reference

## Prepare environment

### Initialize Senzing configuration

Run [senzing-G2ConfigMgr-reference.ipynb](senzing-G2ConfigMgr-reference.ipynb)
to install a Senzing Engine configuration in the database.

### Initialize python environment

In [ ]:
import os
import sys
import json

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing, 
but helps visualize JSON.

In [ ]:
class RenderJSON():
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Engine.

In [ ]:
%run senzing-init-config.ipynb

In [ ]:
%store -r senzing_config_json
%store -r config_id_bytearray

In [ ]:
RenderJSON(senzing_config_json)

## G2Engine

In [ ]:
from senzing import G2Engine, G2Exception

### G2Engine initialization

To start using Senzing G2Engine, create and initialize an instance.
This should be done once per process.
###### Parameters

- **module_name:** (str) A short name given to this instance of the G2Engine
  object.
- **senzing_config_json:** A JSON string containing configuration parameters.
- **verbose_logging:** A boolean which enables diagnostic logging.

Calling this function will return "0" upon success.

In [ ]:
g2_engine = G2Engine()
try:
    return_code = g2_engine.init(
        module_name,
        senzing_config_json,
        verbose_logging)

except G2Exception as err:
    print(g2_engine.getLastException())

#### destroy()
destroy the engine so that we can initalize differently

In [ ]:
g2_engine.destroy()

### initWithConfigID

Alternatively `initWithConfigID()` can be used to specify a configuration.

###### Parameters

- **module_name:** (str) A short name given to this instance of the G2Engine
  object.
- **senzing_config_json:** (str) A JSON string containing configuration parameters.
- **config_id_bytearray:** (bytearray) The configID that you want to initialize the engine with.
- **verbose_logging:** (bool) A boolean which enables diagnostic logging.

In [ ]:
try:
    return_code = g2_engine.initWithConfigID(
        module_name,
        senzing_config_json,
        config_id_bytearray,
        verbose_logging)

except G2Exception as err:
    print(g2_engine.getLastException())

### reinit

The `reinit()` function may be used to reinitialize the engine
using a specified initConfigID.

###### Parameters

- **config_id_bytearray:** (bytearray) The configID that you want to initialize the engine with.

In [ ]:
try:
    return_code = g2_engine.reinit(
        config_id_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())

### primeEngine

The `primeEngine()` method may optionally be called to pre-initialize
some of the heavier weight internal resources of the G2 engine.

In [ ]:
try:
    g2_engine.primeEngine()

except G2Exception as err:
    print(g2_engine.getLastException())

### getActiveConfigID

Call `getActiveConfigID()` to return an identifier for the loaded
Senzing engine configuration.
The call will assign a long integer to a user-designated variable

###### Parameters

- **configuration_id_bytearray:** (bytearray) The identifier value for the engine configuration.
  The result of function call is returned here

In [ ]:
configuration_id_bytearray = bytearray()
try:
    g2_engine.getActiveConfigID(configuration_id_bytearray)
    print("Configuration id: {0}".format(configuration_id_bytearray.decode()))

except G2Exception as err:
    print(g2_engine.getlastException())

### exportConfig

Call `exportConfig()` to retrieve your Senzing engine's configuration.
The call will assign a JSON document to a user-designated buffer,
containing all relevant configuration information
-- the function itself will return "0" upon success.
The exportConfig function accepts the following parameters as input:

###### Parameters

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **config_id_bytearray:** (bytearray) The identifier value for the engine configuration
  can be returned here.

In [ ]:
response_bytearray = bytearray()
config_id_bytearray = bytearray()

try:
    g2_engine.exportConfig(response_bytearray, config_id_bytearray)
    print("Configuration ID: {0}".format(config_id_bytearray.decode()))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### stats

Call `stats()` to retrieve workload statistics for the current process.
These statistics will automatically reset after retrieval.

###### Parameters

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.stats(response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### getRepositoryLastModifiedTime

Call `getRepositoryLastModifiedTime()` to obtain the last modified time of
the Senzing repository,measured in the number of seconds between the last
modified time and January 1, 1970 12:00am GMT (epoch time).
The call will assign a long integer to a user-designated buffer

###### Parameters

- **last_modified_time_bytearray:** (bytearray) The last modified time. The result of function
  call is returned here


In [ ]:
last_modified_time_bytearray = bytearray()

try:
    g2_engine.getRepositoryLastModifiedTime(last_modified_time_bytearray)

# Human readable output.

    from datetime import datetime
    last_modified_unixtime = int(int(last_modified_time_bytearray.decode()) / 1000)
    last_modified_datetime = datetime.fromtimestamp(last_modified_unixtime)

    print("Last modified timestamp: {0}\nLast modified time: {1}"
          .format(last_modified_time_bytearray.decode(), last_modified_datetime))

except G2Exception as err:
    print(g2_engine.getLastException())

### Flags
flags are optional parameters that specify what data about an entity or record to retrive. If no flag parameter is given then the default flags are used, and this will be the practice throughout this notebook but an example is give below

In [ ]:
from senzing import G2EngineFlags

g2_engine_record_flags = G2EngineFlags.G2_RECORD_DEFAULT_FLAGS

load_id = None
response_bytearray = bytearray()
flags_example_datasource_code = "TEST"
flags_example_record_id = "100"
flags_example_data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "John",
        "NAME_FIRST": "Smith",
        "NAME_MIDDLE": "E"
    }],
    "PASSPORT_NUMBER": "PP12345",
    "PASSPORT_COUNTRY": "US",
    "DRIVERS_LICENSE_NUMBER": "DL12345",
    "SSN_NUMBER": "123-45-6789"
}
flags_example_data_as_json = json.dumps(flags_example_data)

try:
    g2_engine.addRecordWithInfo(
        flags_example_datasource_code,
        flags_example_record_id,
        flags_example_data_as_json,
        response_bytearray,
        load_id,
        g2_engine_record_flags)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

## Insert

### Insert parameters

The following variables are used as parameters to the Senzing API.

In [ ]:
datasource_code_1 = "TEST"
record_id_1 = "1"
datasource_code_2 = "TEST"
record_id_2 = "2"
datasource_code_3 = "TEST"
record_id_3 = "3"
datasource_code_4 = "TEST"
record_id_4 = "4"
datasource_code_5 = "TEST"
record_id_5 = "5"
datasource_code_6 = "TEST"
record_id_6 = "6"
datasource_code_7 = "TEST"
record_id_7 = "7"
datasource_code_8 = "TEST"
record_id_8 = "8"

load_id = None

Initial data.

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Smith",
        "NAME_FIRST": "John",
        "NAME_MIDDLE": "M"
    }],
    "PASSPORT_NUMBER": "PP11111",
    "PASSPORT_COUNTRY": "US",
    "DRIVERS_LICENSE_NUMBER": "DL11111",
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

### addRecord

Once the Senzing engine is initialized, use `addRecord()` to load a record
into the Senzing repository
-- `addRecord()` can be called as many times as desired and from multiple
threads at the same time.
The `addRecord()` function returns "0" upon success, and accepts four
parameters as input:

###### Parameters

- **datasource_code:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **record_id:** (str) The record ID, used to identify distinct records
- **data_as_json:** (str) A JSON document with the attribute data for the record
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.

In [ ]:
try:
    datasource_code = datasource_code_1
    record_id = record_id_1
    g2_engine.addRecord(
        datasource_code,
        record_id,
        data_as_json,
        load_id)
    
except G2Exception as err:
    print(g2_engine.getLastException())

### addRecordWithReturnedRecordID

Alternatively `addRecordWithReturnedRecordID()` can be used to return the record ID

###### Parameters

- **datasource_code:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **record_id:** (str) The record ID, used to identify distinct records
- **data_as_json:** (str) A JSON document with the attribute data for the record
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.

In [ ]:
response = bytearray()
try:
    g2_engine.addRecordWithReturnedRecordID(
        datasource_code,
        response_bytearray,
        data_as_json,
        load_id)
    
except G2Exception as err:
    print(g2_engine.getLastException())
print(response_bytearray.decode())

### addRecordWithInfo

Use if you would like to know what resolved entities were modified when
adding the new record.
It behaves identically to addRecord(),
but returns a json document containing the IDs of the affected entities.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify distinct records
- **data_as_json:** (str) A JSON document with the attribute data for the record
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.
- **g2_engine_flags:** (int **[optional]** ) Control flags for specifying what data about the
  entity to retrieve


In [ ]:
response_bytearray = bytearray()
try:
    g2_engine.addRecordWithInfo(
        datasource_code,
        record_id,
        data_as_json,
        response_bytearray,
        load_id)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### addRecordWithInfoWithReturnedRecordID


###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_response:** (bytearray) The record ID of the added record.
- **data_as_json:** (str) A JSON document with the attribute data for the record
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.
- **g2_engine_flags:** (int **[optional]** ) Control flags for specifying what data about the
  entity to retrieve


In [ ]:
response_bytearray = bytearray()
record_response = bytearray()
try:
    g2_engine.addRecordWithInfoWithReturnedRecordID(
        datasource_code,
        data_as_json,
        record_response,
        response_bytearray,
        load_id)

except G2Exception as err:
    print(g2_engine.getLastException())
print(record_response.decode())

## Replace

### replaceRecord

Use the `replaceRecord()` function to update or replace a record in the data
repository.
If record doesn't exist, a new record is added to the data repository.
Like the above functions, `replaceRecord()` returns "0" upon success,
and it can be called as many times as desired and from multiple threads at
the same time.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system
- **record_id:** (str) The record ID, used to identify distinct records
- **data_as_json:** (str) A JSON document with the attribute data for the record
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Miller",
        "NAME_FIRST": "John",
        "NAME_MIDDLE": "M"
    }],
    "PASSPORT_NUMBER": "PP11111",
    "PASSPORT_COUNTRY": "US",
    "DRIVERS_LICENSE_NUMBER": "DL11111",
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    g2_engine.replaceRecord(
        datasource_code,
        record_id,
        data_as_json,
        load_id)

except G2Exception as err:
    print(g2_engine.getLastException())

### replaceRecordWithInfo

`replaceRecordWithInfo()` is available if you would like to know what
resolved entities were modified when replacing a record.
It behaves identically to `replaceRecord()`,
but also returns a json document containing the IDs of the affected entities.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify distinct records
- **data_as_json:** (str) A JSON document with the attribute data for the record
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Jones",
        "NAME_FIRST": "John",
        "NAME_MIDDLE": "M"
    }],
    "PASSPORT_NUMBER": "PP11111",
    "PASSPORT_COUNTRY": "US",
    "DRIVERS_LICENSE_NUMBER": "DL11111",
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)
response_bytearray = bytearray()

try:
    g2_engine.replaceRecordWithInfo(
        datasource_code,
        record_id,
        data_as_json,
        response_bytearray,
        load_id)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

## Re-evaluate

### reevaluateRecord

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify distinct records
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
try:
    g2_engine.reevaluateRecord(
        datasource_code,
        record_id,)

except G2Exception as err:
    print(g2_engine.getLastException())

### reevaluateRecordWithInfo

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify distinct records
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.reevaluateRecordWithInfo(
        datasource_code,
        record_id,
        response_bytearray,)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### reevaluateEntity

###### Parameters

- **entity_id:** (int) The entity ID for the entity to be analyzed
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

First get an entity

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_1 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Re-evaluate the entity.

In [ ]:
try:
    entity_id = entity_id_1
    g2_engine.reevaluateEntity(entity_id)

except G2Exception as err:
    print(g2_engine.getLastException())

### reevaluateEntityWithInfo

###### Parameters

- **entity_id:** (int) The entity ID for the entity to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for - **later. Warning:** This is not a string
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
response_bytearray = bytearray()

try:
    entity_id = entity_id_1
    g2_engine.reevaluateEntityWithInfo(
        entity_id,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

## Redo Processing

Redo records are automatically created by Senzing when certain conditions
occur where it believes more processing may be needed.
Some examples:

- A value becomes generic and previous decisions may need to be revisited
- Clean up after some record deletes
- Detected related entities were being changed at the same time
- A table inconsistency exists, potentially after a non-graceful shutdown

First we will need to have a total of 6 data sources so let's add 5 more.

Create Record and Entity for redo 

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Owens",
        "NAME_FIRST": "Lily"
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_4
    record_id = record_id_4
    g2_engine.addRecord(
        datasource_code,
        record_id,
        data_as_json)

except G2Exception as err:
    print(g2_engine.getLastException())

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_6 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]
    print("Entity ID: {0}".format(entity_id_6))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Create another Record and Entity

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Bauler",
        "NAME_FIRST": "August",
        "NAME_MIDDLE": "E"
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_5
    record_id = record_id_5
    g2_engine.addRecord(
        datasource_code,
        record_id,
        data_as_json)

except G2Exception as err:
    print(g2_engine.getLastException())

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_7 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]
    print("Entity ID: {0}".format(entity_id_7))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Create another Record and Entity

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Barcy",
        "NAME_FIRST": "Brian",
        "NAME_MIDDLE": "H"
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_6
    record_id = record_id_6
    g2_engine.addRecord(
        datasource_code,
        record_id,
        data_as_json)

except G2Exception as err:
    print(g2_engine.getLastException())

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_8 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]
    print("Entity ID: {0}".format(entity_id_8))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Create another Record and Entity

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Miller",
        "NAME_FIRST": "Jack",
        "NAME_MIDDLE": "H"
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_7
    record_id = record_id_7
    g2_engine.replaceRecord(
        datasource_code,
        record_id,
        data_as_json)

except G2Exception as err:
    print(g2_engine.getLastException())

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_9 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]
    print("Entity ID: {0}".format(entity_id_9))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Create another Record and Entity

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Caryn",
        "NAME_FIRST": "Madelynn",
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_8
    record_id = record_id_8
    g2_engine.addRecord(
        datasource_code,
        record_id,
        data_as_json)

except G2Exception as err:
    print(g2_engine.getLastException())

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_10 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]
    print("Entity ID: {0}".format(entity_id_10))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### countRedoRecords

Once the Senzing engine is initialized, use `countRedoRecords()`
to return the remaining internally queued maintenance records in the
Senzing repository.
`countRedoRecords()` takes no arguments and returns <0 for errors.

In [ ]:
return_code = g2_engine.countRedoRecords()
RenderJSON(return_code)

### getRedoRecord

Once the Senzing engine is initialized, 
use `getRedoRecord()` to retrieve the next internally queued maintenance
record into the Senzing repository
-- `getRedoRecord()` can be called as many times as desired and from multiple
threads at the same time but all threads are required to be in the same
process.
`getRedoRecord()` should not be called from multiple processes.
Unlike `processRedoRecord()`, `getRedoRecord()` does not actually process the
record.
To process the record, you would use the G2Engine `process()` function.
The `getRedoRecord()` function returns "0" upon success and an empty response
if there is nothing to do.

###### Parameters

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getRedoRecord(response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
    
RenderJSON(response_bytearray)

### processWithInfo

###### Parameters

- **response_bytearray:** (bytearray) the input record to add
- **process_response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
if (response_bytearray):
    try:
        process_response_bytearray = bytearray()

        g2_engine.processWithInfo(
            response_bytearray.decode(),
            process_response_bytearray)

    except G2Exception as err:
        print(g2_engine.getLastException())

RenderJSON(process_response_bytearray)

### process

###### Parameters

- **response_bytearray:** (bytearray) the input record to add

In [ ]:
if (response_bytearray):
    try:
        g2_engine.process(response_bytearray.decode())

    except G2Exception as err:
        print(g2_engine.getLastException())

### processWithResponse

###### Parameters

- **response_bytearray:** (bytearray) the input record to add
- **response:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
if (response_bytearray):
    response = bytearray()
    try:
        g2_engine.processWithResponse(response_bytearray.decode(), response)

    except G2Exception as err:
        print(g2_engine.getLastException())
RenderJSON(response)

### processRedoRecord

This processes the next redo record and returns it
(If `processRedoRecord()` "response" returns 0
and "response_bytearray" is blank then there are no more redo records to
process and if you do `count.RedoRecords()` again it will return 0)
Has potential to create more redo records in certian situations.

###### Parameters

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
response_bytearray = bytearray()
try:
    g2_engine.processRedoRecord(response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
    
RenderJSON(response_bytearray)

### processRedoRecordWithInfo

`processRedoRecordWithInfo()` is available if you would like to know what
resolved entities were modified when processing a redo record.
It behaves identically to `processRedoRecord()`,
but also returns a json document containing the IDs of the affected entities.
It accepts the following parameters:

###### Parameters

- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **info_bytearray:** (bytearray) Returns the "withInfo" section 

In [ ]:
response_bytearray = bytearray()
info_bytearray = bytearray()

try:
    g2_engine.processRedoRecordWithInfo(
        response_bytearray,
        info_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())

# Pretty-print XML.

RenderJSON(response_bytearray)

# Pretty-print JSON

RenderJSON(info_bytearray)

## Delete

### deleteRecord

Use `deleteRecord()` to remove a record from the data repository
(returns "0" upon success);
`deleteRecord()` can be called as many times as desired and from multiple
threads at the same time.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify distinct records
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.

In [ ]:
try:
    datasource_code = datasource_code_1
    record_id = record_id_1
    g2_engine.deleteRecord(datasource_code, record_id, load_id)

except G2Exception as err:
    print(g2_engine.getLastException())

### deleteRecordWithInfo

`deleteRecordWithInfo()` behaves the same as `deleteRecord()`
but also returns a json document containing the IDs of the affected entities.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with. This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify distinct records.
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **load_id:** (str **[optional]** ) the observation load ID for the record;
  value can be null.
- **g2_engine_flags:** (int **[optioanl]** ) An integer specifying which entity details should be
  included in the export.

In [ ]:
response_bytearray = bytearray()

try:
    datasource_code = datasource_code_2
    record_id = record_id_2
    g2_engine.deleteRecordWithInfo(
        datasource_code,
        record_id,
        response_bytearray,
        load_id,)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Attempt to get the record again.
It should error and give an output similar to "Unknown record".

In [ ]:
try:
    response_bytearray = bytearray()

    datasource_code = datasource_code_1
    record_id = record_id_1
    return_code = g2_engine.getRecord(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    response = json.dumps(response_dictionary, sort_keys=True, indent=4)
    print("Return Code: {0}\n{1}".format(return_code, response))
except G2Exception as err:
    print("Exception: {0}".format(err))

Add record back for later API calls

In [ ]:
try:
    datasource_code = datasource_code_1
    record_id = record_id_1
    g2_engine.addRecord(
        datasource_code,
        record_id,
        data_as_json,
        load_id)
    
except G2Exception as err:
    print(g2_engine.getLastException())

## Search

### Record search

#### getRecord

Use `getRecord()` to retrieve a single record from the data repository;
the record is assigned in JSON form to a user-designated buffer.
Once the Senzing engine is initialized,
`getRecord()` can be called as many times as desired and from multiple
threads at the same time.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The record ID, used to identify the record for retrieval
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for specifying what data about the record to retrieve.

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getRecord(
        datasource_code,
        record_id,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### Entity Search

#### getEntityByRecordID

Use `getEntityByRecordID()` to retrieve entity data based on the ID of a
resolved identity.

###### Parameters

- **datasource_code:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **record_id:** (str) The numeric ID of a resolved entity
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for specifying what data about the entity to retrieve.

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_1 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

#### getEntityByEntityID

Use `getEntityByEntityID()` to retrieve entity data based on the ID of a
resolved identity.

###### Parameters

- **entity_id:** (int) The numeric ID of a resolved entity
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for specifying what data about the entity to retrieve.

In [ ]:
response_bytearray = bytearray()

try:
    entity_id = entity_id_1
    g2_engine.getEntityByEntityID(
        entity_id,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### Search By Attributes

#### searchByAttributes

This function is similar but preferable to the searchByAttributes() function.

Use `searchByAttributes()` to retrieve entity data based on
a user-specified set of entity attributes.

###### Parameters

- **data_as_json:** (str) A JSON document with the attribute data to search for.
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.searchByAttributes(
        data_as_json,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### Finding Paths

First you will need to create some records so that you have some that you can
compare.
Can you see what is the same between this record and the previous one?

#### Create records for paths

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Miller",
        "NAME_FIRST": "Max",
        "NAME_MIDDLE": "W"
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_2
    record_id = record_id_2
    g2_engine.replaceRecord(
        datasource_code,
        record_id,
        data_as_json)

except G2Exception as err:
    print(g2_engine.getLastException())

Replace values for Record #3

In [ ]:
data = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Miller",
        "NAME_FIRST": "Mildred"
    }],
    "SSN_NUMBER": "111-11-1111"
}
data_as_json = json.dumps(data)

try:
    datasource_code = datasource_code_3
    record_id = record_id_3
    g2_engine.replaceRecord(
        datasource_code,
        record_id,
        data_as_json,
        None)

except G2Exception as err:
    print(g2_engine.getLastException())

Locate "entity identifier" for Record #1

In [ ]:
response_bytearray = bytearray()

try:
    datasource_code = datasource_code_1
    record_id = record_id_1
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_1 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]

    print("Entity ID: {0}".format(entity_id_1))

except G2Exception as err:
    print(g2_engine.getlastException())

Locate "entity identifier" for Record #2

In [ ]:
response_bytearray = bytearray()

try:
    datasource_code = datasource_code_2
    record_id = record_id_2
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_2 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]

    print("Entity ID: {0}".format(entity_id_2))

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

Locate "entity identifier" for Record #3

In [ ]:
response_bytearray = bytearray()

try:
    datasource_code = datasource_code_3
    record_id = record_id_3
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id_3 = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]

    print("Entity ID: {0}".format(entity_id_3))

except G2Exception as err:
    print(g2_engine.getlastException())
RenderJSON(response_bytearray)

#### findPathByEntityID

The `findPathByEntityID()` function can be used to find single relationship paths between two entities. Paths are found using known relationships with other entities.

###### Parameters

- **entity_id_start:** (int) The entity ID for the starting entity of the search path
- **entity_id_end:** (int) The entity ID for the ending entity of the search path
- **max_degree:** (int) The number of relationship degrees to search
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

The functions return a JSON document that identifies the path between the
entities,
and the information on the entities in question.
The document contains a section called "ENTITY_PATHS" which gives
the path from one entity to the other.

In [ ]:
# Define search variables.

max_degree = 3

# Find the path by entity ID.

response_bytearray = bytearray([])

try:
    entity_id_start = entity_id_2
    entity_id_end = entity_id_3
    g2_engine.findPathByEntityID(
        entity_id_start,
        entity_id_end,
        max_degree,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

#### findPathByRecordID

The `findPathByRecordID()` function
can be used to find single relationship paths between two entities.
Paths are found using known relationships with other entities.

###### Parameters

- **datasource_code_start:** (str) The data source for the starting entity of the
  search path
- **datasource_code_end:** (str) The data source for the ending entity of the search
  path
- **record_id_start:** (int) The record ID for the starting entity of the search path
- **record_id_end:** (int) The record ID for the ending entity of the search path
- **max_degree:** (int) The number of relationship degrees to search
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

The functions return a JSON document that identifies the path between the
entities,
and the information on the entities in question.
The document contains a section called "ENTITY_PATHS" which gives
the path from one entity to the other.

In [ ]:
# Define search variables.

max_degree = 3

# Find the path by record ID.

response_bytearray = bytearray([])

try:
    datasource_code_start = datasource_code_2
    datasource_code_end = datasource_code_3
    record_id_start = record_id_2
    record_id_end = record_id_3
    g2_engine.findPathByRecordID(
        datasource_code_start, record_id_start,
        datasource_code_end, record_id_end,
        max_degree,
        
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getlastException())
# Print the results.

RenderJSON(response_bytearray)

#### findPathExcludingByEntityID

The `findPathExcludingByEntityID()` function can be used to find single relationship paths between two
entities.
Paths are found using known relationships with other entities.
In addition, it will find paths that exclude certain entities from being on
the path.


When excluding entities, the user may choose to either (a) strictly exclude
the entities,
or (b) prefer to exclude the entities, but still include them if no other
path is found.
By default, entities will be strictly excluded.
A "preferred exclude" may be done by specifying the
`G2_FIND_PATH_PREFER_EXCLUDE` control flag.

###### Parameters

- **entity_id_start:** (int) The entity ID for the starting entity of the search path
- **entity_id_end:** (int) The entity ID for the ending entity of the search path
- **max_degree:** (int) The number of relationship degrees to search
- **excluded_entities_as_json:** (str) Entities that should be avoided on the path
  (JSON document)
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags

In [ ]:
# Define search variables.

max_degree = 4
excluded_entities = {
    "ENTITIES": [{
        "ENTITY_ID": entity_id_1
    }]}
excluded_entities_as_json = json.dumps(excluded_entities)

# Find the path by entity ID.

response_bytearray = bytearray([])

try:
    entity_id_start = entity_id_2
    entity_id_end = entity_id_3
    g2_engine.findPathExcludingByEntityID(
        entity_id_start,
        entity_id_end,
        max_degree,
        excluded_entities_as_json,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

#### findPathExcludingByRecordID

The `findPathExcludingByRecordID()`
function can be used to find single relationship paths between two
entities.
Paths are found using known relationships with other entities.
In addition, it will find paths that exclude certain entities from being on
the path.

When excluding entities, the user may choose to either (a) strictly exclude
the entities,
or (b) prefer to exclude the entities, but still include them if no other
path is found.
By default, entities will be strictly excluded.
A "preferred exclude" may be done by specifying the
`G2_FIND_PATH_PREFER_EXCLUDE` control flag.

###### Parameters

- **datasource_code_start:** (str) The data source for the starting entity of the
  search path
- **datasource_code_end:** (str) The data source for the ending entity of the search
  path
- **record_id_start:** (str) The record ID for the starting entity of the search path
- **record_id_end:** (str) The record ID for the ending entity of the search path
- **max_degree:** (int) The number of relationship degrees to search
- **excluded_entities_as_json:** (str) Entities that should be avoided on the path
  (JSON document)
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags


In [ ]:
# Define search variables.

excluded_records = {
    "RECORDS": [{
        "RECORD_ID": record_id_1,
        "DATA_SOURCE": datasource_code_1
    }]}
excluded_records_as_json = json.dumps(excluded_records)

# Find the path by record ID.

response_bytearray = bytearray([])

try:
    datasource_code_start = datasource_code_2
    datasource_code_end = datasource_code_3
    record_id_start = record_id_2
    record_id_end = record_id_3
    g2_engine.findPathExcludingByRecordID(
        datasource_code_start, record_id_start,
        datasource_code_end, record_id_end,
        max_degree,
        excluded_records_as_json,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

### Finding Paths with Required Sources

#### findPathIncludingSourceByEntityID

The `findPathIncludingSourceByEntityID()` function
can be used to find single relationship paths between two entities.
In addition, one of the enties along the path must include a specified data
source.

Specific entities may also be excluded, using the same methodology as the
`findPathExcludingByEntityID()` and `findPathExcludingByRecordID()`
functions use.

###### Parameters

- **entity_id_start:** (int) The entity ID for the starting entity of the search path
- **entity_id_end:** (int) The entity ID for the ending entity of the search path
- **max_degree:** (int) The number of relationship degrees to search
- **excluded_entities_as_json:** (str) Entities that should be avoided on the path
  (JSON document)
- **required_dsrcs_as_json:** (str) Entities that should be avoided on the path
  (JSON document)
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags


In [ ]:
# Define search variables.

max_degree = 4
excluded_entities = {
    "ENTITIES": [{
        "ENTITY_ID": entity_id_1
    }]}
excluded_entities_as_json = json.dumps(excluded_entities)
required_dsrcs = {
    "DATA_SOURCES": [
        datasource_code_1
    ]}
required_dsrcs_as_json = json.dumps(excluded_entities)

# Find the path by entity ID.

response_bytearray = bytearray([])

try:
    entity_id_start = entity_id_2
    entity_id_end = entity_id_3
    g2_engine.findPathIncludingSourceByEntityID(
        entity_id_start,
        entity_id_end,
        max_degree,
        excluded_entities_as_json,
        required_dsrcs_as_json,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

#### findPathIncludingSourceByRecordID


The 
`findPathIncludingSourceByRecordID()` function
can be used to find single relationship paths between two entities.
In addition, one of the enties along the path must include a specified data
source.

Specific entities may also be excluded, using the same methodology as the
`findPathExcludingByEntityID()` and `findPathExcludingByRecordID()`
functions use.

###### Parameters

- **datasource_code_start:** (str) The data source for the starting entity of the
  search path
- **datasource_code_end:** (str) The data source for the ending entity of the search
  path
- **record_id_start:** (str) The record ID for the starting entity of the search
  path
- **record_id_end:** (str) The record ID for the ending entity of the search path
- **max_degree:** (int) The number of relationship degrees to search
- **excluded_entities_as_json:** (str) Entities that should be avoided on the path
  (JSON document)
- **required_dsrcs_as_json:** (str) Entities that should be avoided on the path
  (JSON document)
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags


In [ ]:
# Define search variables.

excluded_records = {
    "RECORDS": [{
        "RECORD_ID": record_id_1,
        "DATA_SOURCE": datasource_code_1
    }]}
excluded_records_as_json = json.dumps(excluded_records)

# Find the path by record ID.

response_bytearray = bytearray([])

try:
    datasource_code_start = datasource_code_2
    datasource_code_end = datasource_code_3
    record_id_start = record_id_2
    record_id_end = record_id_3
    g2_engine.findPathIncludingSourceByRecordID(
        datasource_code_start, record_id_start,
        datasource_code_end, record_id_end,
        max_degree,
        excluded_records_as_json,
        required_dsrcs_as_json,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

### Finding Networks

The `findNetworkByEntityID()` and `findNetworkByRecordID()` functions
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

Entities can be searched for by either Entity ID or by Record ID,
depending on which function is chosen.

These functions have the following parameters:

- **entity_list_as_json:** A list of entities, specified by Entity ID
  (JSON document)
- **record_list_as_json:** A list of entities, specified by Record ID
  (JSON document)
- **max_degree:** The maximum number of degrees in paths between search
  entities
- **buildout_degree:** The number of degrees of relationships to show around
  each search entity
- **max_entities:** The maximum number of entities to return in the
  discovered network

They also have various arguments used to return response documents

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

#### findNetworkByEntityID

The `findNetworkByEntityID()` function
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

###### Parameters

- **max_degree:** (int) The maximum number of degrees in paths between search
  entities
- **buildout_degree:** (int) The number of degrees of relationships to show around
  each search entity
- **max_entities:** (int) The maximum number of entities to return in the
  discovered network
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags


The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

In [ ]:
# Define search variables.

entity_list = {
    "ENTITIES": [{
        "ENTITY_ID": entity_id_1
    }, {
        "ENTITY_ID": entity_id_2
    }, {
        "ENTITY_ID": entity_id_3
    }]}
entity_list_as_json = json.dumps(entity_list)
max_degree = 2
buildout_degree = 1
max_entities = 12

# Find the network by entity ID.

response_bytearray = bytearray()

try:
    g2_engine.findNetworkByEntityID(
        entity_list_as_json,
        max_degree, 
        buildout_degree,
        max_entities,
        
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

#### findNetworkByRecordID

The `findNetworkByRecordID()` function
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

###### Parameters

- **record_list_as_json:** (str) A list of entities, specified by Record ID
  (JSON document)
- **max_degree:** (int) The maximum number of degrees in paths between search
  entities
- **buildout_degree:** (int) The number of degrees of relationships to show around
  each search entity
- **max_entities:** (int) The maximum number of entities to return in the
  discovered network
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags


The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

In [ ]:
# Define search variables.

record_list = {
    "RECORDS": [{
        "RECORD_ID": record_id_1,
        "DATA_SOURCE": datasource_code_1
    }, {
        "RECORD_ID": record_id_2,
        "DATA_SOURCE": datasource_code_2
    }, {
        "RECORD_ID": record_id_3,
        "DATA_SOURCE": datasource_code_3
    }]}
record_list_as_json = json.dumps(record_list)

# Find the network by record ID.

response_bytearray = bytearray()

try:
    g2_engine.findNetworkByRecordID(
        record_list_as_json,
        max_degree,
        buildout_degree,
        max_entities,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
# Print the results.

RenderJSON(response_bytearray)

## Connection Details

### findInterestingEntitiesByEntityID

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.findInterestingEntitiesByEntityID(
        entity_id_3,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### findInterestingEntitiesByRecordID

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.findInterestingEntitiesByRecordID(
        datasource_code_1,
        record_id_1,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

The `whyEntities()`, `whyRecords()`, `whyEntityByRecordID()`,`whyEntityByEntityID()`, and `howEntityByEntityID()` functions can be used
to determine why and how records belong to their resolved entities.
These functions will compare the record data within an entity against the
rest of the entity data, and show why they are connected.
This is calculated based on the features that record data represents.

###### Parameters

- **datasource_code:** (str) The data source for the record to be analyzed
- **entity_id:** (int) The entity ID for the entity to be analyzed
- **record_id:** (str) The record ID for the record to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Operational flags

The function returns a JSON document that gives the results of the record
analysis.
The document contains a section called "WHY_RESULTS",
which shows how specific records relate to the rest of the entity.
It has a "WHY_KEY", which is similar to a match key, in defining the relevant
connected data.
It shows candidate keys for features that initially cause the records
to be analyzed for a relationship,
plus a series of feature scores that show how similar the feature data was.

The response document also contains a separate ENTITIES section,
with the full information about the resolved entity.
(Note: When working with this entity data,
Senzing recommends using the flags `G2_ENTITY_OPTION_INCLUDE_INTERNAL_FEATURES`
and `G2_ENTITY_OPTION_INCLUDE_FEATURE_STATS`.
This will provide detailed feature data that is not included by default,
but is useful for understanding the WHY_RESULTS data.)

### whyEntities


###### Parameters

- **entity_id_1:** (int) The entity ID for the first entity to be analyzed
- **entity_id_2:** (int) The entity ID for the second entity to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.whyEntities(
        entity_id_1,
        entity_id_2,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### whyRecords

###### Parameters

- **datasource_code_1:** (str) The name of the first data source the record
  is associated with.
  This value is configurable to the system
- **record_id_1:** (int) The record ID for the first record to be analyzed
- **datasource_code_2:** (str) The name of the second data source the record
  is associated with.
  This value is configurable to the system
- **record_id_1:** (int) The record ID for the second record to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.whyRecords(
        datasource_code_1,
        record_id_1,
        datasource_code_2,
        record_id_2,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### whyEntityByRecordID

###### Parameters

- **datasource_code:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **record_id:** (int) The record ID for the record to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.whyEntityByRecordID(
        datasource_code_1,
        record_id_1,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### whyEntityByEntityID

###### Parameters

- **entity_id:** (int) The entity ID for the entity to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

In [ ]:
response_bytearray = bytearray()

try:
    entity_id = entity_id_1
    g2_engine.whyEntityByEntityID(
        entity_id,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

### howEntityByEntityID

###### Parameters

- **entity_id:** (int) The entity ID for the entity to be analyzed
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*
- **g2_engine_flags:** (int **[optional]**) Control flags for outputting entities

Create some records to show resolution steps

In [ ]:
data1 = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Schmoe",
        "NAME_FIRST": "Joe",
        "NAME_MIDDLE": "M"
    }],
    "SSN_NUMBER": "111-22-3456",
    "DOB": "03/22/1982"
}
data2 = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Schmoe",
        "NAME_FIRST": "Joe",
        "NAME_MIDDLE": "M"
    }],
    "SSN_NUMBER": "111-22-3456",
    "CITIZENSHIP": "British"
}
data3 = {
    "NAMES": [{
        "NAME_TYPE": "PRIMARY",
        "NAME_LAST": "Schmoe",
        "NAME_FIRST": "Joe",
        "NAME_MIDDLE": "M"
    }],
    "SSN_NUMBER": "111-22-3456",
    "NATIONALITY": "Australian"
}
data_as_json1 = json.dumps(data1)
data_as_json2 = json.dumps(data2)
data_as_json3 = json.dumps(data3)

record_id1 = "10"
record_id2 = "11"
record_id3 = "12"
datasource_code = "TEST"
g2_engine.addRecord(datasource_code,record_id1,data_as_json1,load_id)
g2_engine.addRecord(datasource_code,record_id2,data_as_json2,load_id)
g2_engine.addRecord(datasource_code,record_id3,data_as_json3,load_id)

response_bytearray = bytearray()

try:
    g2_engine.getEntityByRecordID(
        datasource_code,
        record_id1,
        response_bytearray)

    response_dictionary = json.loads(response_bytearray)
    entity_id = response_dictionary["RESOLVED_ENTITY"]["ENTITY_ID"]

except G2Exception as err:
    print(g2_engine.getLastException())

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.howEntityByEntityID(
        entity_id,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray.decode())

## Reporting

Exporting entity data from resolved entities is one of the core purposes of
Senzing software.
In just a few short steps, the Senzing engine allows users to export entity
data in either JSON or CSV format.

### exportJSONEntityReport

There are three steps to exporting resolved entity data from the G2Engine
object in JSON format.
First, use the `exportJSONEntityReport()` method to generate the `export_handle`.

###### Parameters

- **g2_engine_flags**: (int **[optional]**) An integer specifying which entity details should be
  included in the export.

In [ ]:
try:
    export_handle = g2_engine.exportJSONEntityReport()

except G2Exception as err:
    print(g2_engine.getLastException())

### fetchNext

Second, use the `fetchNext()` method to read the exportHandle and export a
row of JSON output containing the entity data for a single entity.
Note that successive calls of `fetchNext()` will export successive rows of
entity data.

###### Parameters

- **export_handle:** (int) A long integer from which resolved entity data may be
  read and exported.
- **response_bytearray:** (bytearray) Object to store the output of the method. Can be used to print or store for later. **Warning:** This is not a *string*

In [ ]:
try:
    while True:
        response_bytearray = bytearray()
        g2_engine.fetchNext(export_handle, response_bytearray)
        if not response_bytearray:
            break
        response_dictionary = json.loads(response_bytearray)
        response = json.dumps(response_dictionary, sort_keys=True, indent=4)
        print(response)
        
except G2Exception as err:
    print(g2_engine.getLastException())

### closeExport

###### Parameters

- **export_handle:** (int) A long integer from which resolved entity data may be
  read and exported.

In [ ]:
try:
    g2_engine.closeExport(export_handle)

except G2Exception as err:
    print(g2_engine.getLastException())

### exportCSVEntityReport

There are three steps to exporting resolved entity data from the G2Engine
object in CSV format.
First, use the `exportCSVEntityReport()` method to generate a long integer,
referred to here as an 'export_handle'.

###### Parameters

- **csv_column_list:** (str) A comma-separated list of column names for the CSV
  export. (These are listed a little further down.)
- **g2_engine_flags:** (int **[optional]**) An integer specifying which entity details should be
  included in the export.
  See the "Entity Export Flags" section in the link below for further details.

Second, use the `fetchNext()` method to read the exportHandle and export a
row of CSV output containing the entity data for a single entity.
Note that the first call of `fetchNext()` will yield a header row,
and that successive calls of `fetchNext()` will export successive rows of
entity data.

In [ ]:
try:
    headers = 'RESOLVED_ENTITY_ID,RESOLVED_ENTITY_NAME,RELATED_ENTITY_ID,MATCH_LEVEL,MATCH_KEY,IS_DISCLOSED,IS_AMBIGUOUS,DATA_SOURCE,RECORD_ID,JSON_DATA,LAST_SEEN_DT,NAME_DATA,ATTRIBUTE_DATA,IDENTIFIER_DATA,ADDRESS_DATA,PHONE_DATA,RELATIONSHIP_DATA,ENTITY_DATA,OTHER_DATA'
    export_handle = g2_engine.exportCSVEntityReport(headers)

    while True:
        response_bytearray = bytearray()
        g2_engine.fetchNext(export_handle, response_bytearray)
        if not response_bytearray:
            break
        print(response_bytearray.decode())

    g2_engine.closeExport(export_handle)

except G2Exception as err:
    print(g2_engine.getLastException())

## Cleanup

To purge the G2 repository, use the aptly named `purgeRepository()` method.
This will remove every record in your current repository.

### purgeRepository

In [ ]:
try:
    g2_engine.purgeRepository()

except G2Exception as err:
    print(g2_engine.getLastException())

### destroy

Once all searching is done in a given process,
call `destroy()` to uninitialize Senzing and clean up resources.
You should always do this once at the end of each process.

In [ ]:
try:
    g2_engine.destroy()

except G2Exception as err:
    print(g2_engine.getLastException)